## Make Calls to Yahoo Finance ##

In [5]:
# libraries
import pandas as pd
import yfinance as YF


In [32]:
def get_tickers(tickers=[], period='1d'):
    # get tickers from yahoo finance
    table = pd.DataFrame()
    for ticker in tickers:
        data = YF.Ticker(ticker)
        df = data.history(period=period)
        df['ticker'] = ticker
        table = pd.concat([table, df])
    return table

In [42]:
tickers = ['AAPL', 'PLTR', 'WMT', 'META', 'AMZN', 'LMT']
stocks = get_tickers(tickers, period='max')
stocks.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 47952 entries, 1980-12-12 00:00:00-05:00 to 2025-03-27 00:00:00-04:00
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open          47952 non-null  float64
 1   High          47952 non-null  float64
 2   Low           47952 non-null  float64
 3   Close         47952 non-null  float64
 4   Volume        47952 non-null  int64  
 5   Dividends     47952 non-null  float64
 6   Stock Splits  47952 non-null  float64
 7   ticker        47952 non-null  object 
dtypes: float64(6), int64(1), object(1)
memory usage: 3.3+ MB


In [19]:
# test yf
data = YF.Ticker("PLTR")
df = data.history(period="5d")
print(df.head())

                                Open       High        Low      Close  \
Date                                                                    
2025-03-21 00:00:00-04:00  85.849998  91.070000  84.459999  90.959999   
2025-03-24 00:00:00-04:00  93.400002  97.389999  92.279999  96.750000   
2025-03-25 00:00:00-04:00  97.220001  98.169998  95.080002  96.500000   
2025-03-26 00:00:00-04:00  96.290001  97.300003  90.820000  92.279999   
2025-03-27 00:00:00-04:00  91.014999  93.650002  89.669998  90.089996   

                              Volume  Dividends  Stock Splits  
Date                                                           
2025-03-21 00:00:00-04:00  116748700        0.0           0.0  
2025-03-24 00:00:00-04:00  112164700        0.0           0.0  
2025-03-25 00:00:00-04:00   78015300        0.0           0.0  
2025-03-26 00:00:00-04:00   82138300        0.0           0.0  
2025-03-27 00:00:00-04:00   82605300        0.0           0.0  


In [10]:
type(df)

pandas.core.frame.DataFrame